###### import libraries

In [ ]:
import os,sys,rasterio,folium,geopandas,numpy,branca

###### import the STReaM - Water Table Depth - TOPOgraphy (strmwtdtopo) module

In [ ]:
src = r'..\..'
sys.path.append(os.path.abspath(src))
import strmwtdtopo

###### init namelist object

In [ ]:
fname_namelist = '././namelist.txt'
namelist = strmwtdtopo.Namelist(filename=fname_namelist)

###### set directory paths

###### read vector data

In [4]:
domain = geopandas.read_file(fname_domain)
nhdhr = geopandas.read_file(fname_nhdflowlinehr)
soils = geopandas.read_file(fname_soil_texture).to_crs('EPSG:4326')

###### init folium map

In [5]:
domain_centroid = domain.to_crs('+proj=cea').centroid.to_crs(domain.crs)
imap = folium.Map([domain_centroid.y.iloc[0],domain_centroid.x.iloc[0]])
folium_crs = str(imap.crs).replace('EPSG','EPSG:')

###### create symbologies for vector data

In [6]:
nhdfg = folium.FeatureGroup(name='NHD HD')
for _, r in nhdhr.iterrows(): folium.GeoJson(data=geopandas.GeoSeries(r['geometry']).to_json()).add_to(nhdfg)
domainfg = folium.FeatureGroup(name='Domain')
for _, r in domain.iterrows():folium.GeoJson(data=geopandas.GeoSeries(r['geometry']).to_json(),style_function=lambda x:{"color":"black","fillColor":"none"}).add_to(domainfg)
soilshapefg = folium.FeatureGroup(name='Soils')
for _, r in soils.iterrows():folium.GeoJson(data=geopandas.GeoSeries(r['geometry']).to_json(),style_function=lambda x:{"color":"black","fillColor":"none"}).add_to(soilshapefg)

###### add gridded data to map

In [7]:
dtp = {'Transmissivity Decay Factor (f)':[fname_soil_transmissivity_decay_factor,branca.colormap.linear.Greys_07]}
for name, [fname,cmap] in dtp.items():
    if os.path.isfile(fname):
        with rasterio.open(fname,'r') as src:            
            dst_transform, dst_width, dst_height = rasterio.warp.calculate_default_transform(src.crs, folium_crs, src.width, src.height, *src.bounds)
            src_prj_meta = src.meta.copy()
            src_prj_meta.update({'driver':'GTIFF','crs':folium_crs,'transform':dst_transform,'width':dst_width,'height':dst_height,'nodata':numpy.nan,'count':1,'dtype':numpy.float64})
            with rasterio.io.MemoryFile().open(**src_prj_meta) as src_prj:
                rasterio.warp.reproject(source=rasterio.band(src, 1),destination=rasterio.band(src_prj, 1),src_transform=src.transform,src_crs=src.crs,dst_transform=dst_transform,dst_crs=folium_crs,resampling=rasterio.enums.Resampling.nearest)
                vals = src_prj.read(1)
                bbox = rasterio.warp.transform_bounds(src_prj.crs, "EPSG:4326", *src_prj.bounds)
                cmap = cmap.scale(numpy.nanmin(vals),numpy.nanmax(vals)).to_step(6)
                cmap.caption = name
                cmap.add_to(imap)
                cmapf = lambda x : (0, 0, 0, 0) if numpy.isnan(x) else cmap.rgba_floats_tuple(x)
                folium.raster_layers.ImageOverlay(name=name,image=vals,bounds=[[bbox[1],bbox[0]], [bbox[3], bbox[2]]],colormap=cmapf).add_to(imap)

###### add vector data to map

In [8]:
nhdfg.add_to(imap)
domainfg.add_to(imap)
soilshapefg.add_to(imap);

###### add layer control and show map

In [9]:
folium.LayerControl().add_to(imap)
imap